### Работа с ollama моделями через LangChain

In [28]:
import numpy as np
from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings
ollama_url = '192.168.8.86'

In [29]:
llm =  OllamaLLM(model="llama3.2:1b", temperature=1, base_url=f"{ollama_url}:11434")
response = llm.invoke("hi, do u love cats")
print(response)

/var/folders/k0/w5_bdr7169jfn7q2nltd0bz00000gn/T/ipykernel_78369/1096076164.py:1: ResourceWarning: unclosed <socket.socket fd=74, family=2, type=1, proto=6, laddr=('192.168.8.183', 50855), raddr=('192.168.8.86', 11434)>
  llm =  OllamaLLM(model="llama3.2:1b", temperature=1, base_url=f"{ollama_url}:11434")


Hello. I'm just a language model, I don't have personal feelings or emotions, but I can provide information about cats if you're interested! Many people adore cats for their playful personalities, soft fur, and affectionate nature. Do you have a feline friend at home?


In [30]:
embeddings = OllamaEmbeddings(model="all-minilm:33m",  base_url=f"{ollama_url}:11434")

texts = [
    "Что такое LangChain?",
    "Как использовать Ollama для создания эмбеддингов?",
    "Какие модели поддерживает Ollama?"
]

vectors = embeddings.embed_documents(texts)

vectors = np.array(vectors)
new_matr = vectors/np.linalg.norm(vectors, axis=1, keepdims=True)
new_matr @ new_matr.T

/var/folders/k0/w5_bdr7169jfn7q2nltd0bz00000gn/T/ipykernel_78369/1192277074.py:1: ResourceWarning: unclosed <socket.socket fd=75, family=2, type=1, proto=6, laddr=('192.168.8.183', 50856), raddr=('192.168.8.86', 11434)>
  embeddings = OllamaEmbeddings(model="all-minilm:33m",  base_url=f"{ollama_url}:11434")


array([[1.        , 0.19035119, 0.27979231],
       [0.19035119, 1.        , 0.68131581],
       [0.27979231, 0.68131581, 1.        ]])

### Примеры для Weaviate

In [31]:
import weaviate
from weaviate.classes.init import Auth
from weaviate.classes.config import Configure
from weaviate.classes.query import MetadataQuery
import requests, json, os

#### Инициализация клиента

In [32]:
client = weaviate.connect_to_local(
    host=ollama_url,  # Укажите адрес хоста
    port=8080,         # Укажите порт HTTP
    grpc_port=50051    # Укажите порт gRPC
)

print(client.is_ready())

True


#### Коллекции

In [20]:
# Удалить все коллекции
client.collections.delete_all()

In [21]:
# Создать коллекцию с привязанным векторизатором. Два варианта векторизаторов
questions = client.collections.create(
    name="Question",
    #vectorizer_config=Configure.Vectorizer.text2vec_ollama(     # Configure the Ollama embedding integration
    #    api_endpoint="http://ollama:11434",       # Allow Weaviate from within a Docker container to contact your Ollama instance
    #    model="all-minilm:33m",                              # The model to use
    #),
    vectorizer_config =[
        Configure.NamedVectors.text2vec_ollama(
            name="title_vector",
            source_properties=["answer"],
            api_endpoint="http://ollama:11434",  # If using Docker, use this to contact your local Ollama instance
            model="all-minilm:33m",  # The model to use, e.g. "nomic-embed-text"
    )
    ],
    generative_config=Configure.Generative.ollama(              # Configure the Ollama generative integration
        api_endpoint="http://ollama:11434",       # Allow Weaviate from within a Docker container to contact your Ollama instance
        model="llama3.2:1b",                                       # The model to use
    )
)

In [22]:
# Наполнить коллекцию
resp = requests.get(
    "https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json"
)
data = json.loads(resp.text)

questions = client.collections.get("Question")

with questions.batch.dynamic() as batch:
    for d in data:
        batch.add_object({
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        })

In [33]:
# Провести векторный поиск по полю answer
questions = client.collections.get("Question")

response = questions.query.near_text(
    query="Elephant",
    limit=2
)

for obj in response.objects:
    print(json.dumps(obj.properties, indent=2))

{
  "answer": "Elephant",
  "question": "It's the only living mammal in the order Proboseidea",
  "category": "ANIMALS"
}
{
  "answer": "the nose or snout",
  "question": "The gavial looks very much like a crocodile except for this bodily feature",
  "category": "ANIMALS"
}


In [34]:
# Провести векторный поиск и поиск по ключевым словам по полю answer
questions = client.collections.get("Question")

response = questions.query.hybrid(
    query="Elephant",  # The model provider integration will automatically vectorize the query
    limit=2
)

for obj in response.objects:
    print(json.dumps(obj.properties, indent=2))

{
  "answer": "Elephant",
  "question": "It's the only living mammal in the order Proboseidea",
  "category": "ANIMALS"
}
{
  "answer": "the nose or snout",
  "question": "The gavial looks very much like a crocodile except for this bodily feature",
  "category": "ANIMALS"
}


In [35]:
# Найти документ, наиболее близкий по полю "answer" к query, и составить на его основе твит с помощью llama3.2
questions = client.collections.get("Question")

response = questions.generate.near_text(
    query="Elephant",
    limit=2,
    grouped_task="Write a tweet with emojis about these facts."
)

print(response.generated)  # Inspect the generated text

"🐘 Did you know elephants are the only living mammals in the order Proboseidea? 🌟 And, their iconic noses are actually fused to their face! 👃 What else do you wonder about these incredible creatures? #elephantfacts #wildlife #animals"


In [18]:
client.close()